In [1]:
from gridworld import GridWorld
import numpy as np
import time

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
world=\
    """
    wwwwwwwww
    wa      w
    w    ww w
    www     w
    w    w  w
    w      gw
    wwwwwwwww
    """
env=GridWorld(world)
env._max_epi_step=100

# Monti-Carlo policy Evaluation

In [5]:
def get_episode(policy):
    l=[]
    curr_state=env.reset()
    done=False
    while not done:
        a=policy[curr_state]
        next_state,reward,done,info=env.step(a)
        l.append((curr_state,reward))
        curr_state=next_state
    return l

In [6]:
def get_returns_from_episode(episode):
    G=[]
    for i,(s,r) in enumerate(episode):
        summ=r
        future=episode[i+1:]
        for j,(sn,rn) in enumerate(future):
            summ=summ+(0.99**(j+1))*rn
        G.append((s,summ))
    return G

In [ ]:
#method 1

In [19]:
policy=np.random.choice([0,1,2,3],size=29) #policy
experience=[get_returns_from_episode(get_episode(policy)) for i in range(1000)] #experience
G_total=np.zeros(env.state_count)  #G_summ
N_total=np.zeros(env.state_count)  #state seen count

In [21]:
for epi in experience:
    seen=[]
    for s,G in epi:
        if s not in seen:
            G_total[s]+=G
            N_total[s]+=1
            seen.append(s)

V=G_total/N_total

/home/prasenjit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [22]:
V

array([-62.2347953 , -53.64030297, -53.78894552, -43.88601096,
       -43.18154378, -36.72242941, -30.28091744, -61.6506605 ,
       -55.79177446, -54.4736039 , -40.80881467, -24.73450669,
       -38.6257125 , -22.39663307, -20.01122497, -10.79224115,
         1.47759457, -28.82536221, -30.51095084, -39.04643804,
       -31.33381475, -10.95348652,   6.11248325, -29.5822549 ,
       -33.41781299, -38.25521156, -31.60282777, -27.61797959,
       -12.40672958,          nan])

In [23]:
#method 2

In [27]:
policy=np.random.choice([0,1,2,3],size=29) #policy
experience=[get_returns_from_episode(get_episode(policy)) for i in range(1000)] #experience
V=np.zeros(env.state_count)  #G_summ
N=np.zeros(env.state_count)

In [29]:
for epi in experience:
    seen=[]
    for s,G in epi:
        if s not in seen:
            V[s]=V[s]+(1/(N[s]+1))*(G-V[s])
            N[s]+=1
            seen.append(s)

In [30]:
V

array([-6.04529355e+01, -5.98937009e+01, -5.05890224e+01, -3.89393380e+01,
       -3.80474855e+01, -3.71947202e+01, -3.17964901e+01, -5.05352463e+01,
       -5.93411788e+01, -4.52279613e+01, -4.28441891e+01, -2.52176019e+01,
       -3.36077825e+01, -2.97468482e+01, -2.48684994e+01, -1.43355095e+01,
       -1.49299591e+01, -1.34508385e+01, -1.57946417e+01, -2.65228347e+01,
        6.07161603e-03, -9.13763015e+00,  7.93439025e+00, -6.60409046e+00,
       -4.57216318e+00, -5.22174765e-01,  3.26629342e+00,  6.60444817e+00,
        8.21178874e+00,  0.00000000e+00])

# TD Learning

In [31]:
def get_tuple(policy):
    curr_state=env.reset()
    while True:
        a=policy[curr_state]
        next_state,reward,done,info=env.step(a)
        yield(curr_state,reward,next_state)
        if done:
            curr_state=env.reset()
        else:
            curr_state=next_state

In [50]:
gen=get_tuple(policy)

In [53]:
alpha=0.001
gamma=0.99
v=np.zeros(env.state_count)

In [54]:
for step in range(100000):
    s,r,s_prime=next(gen)
    v[s]=v[s]+alpha*((r+gamma*v[s_prime])-v[s])